### <b>Detectron2 설치</b>

In [ ]:
!pip install pyyaml==5.1

# PyTorch 1.9.0 버전에 맞는 Detectron2 설치하기
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

# 설치가 완료되면 런타임 재시작하기

In [ ]:
# PyTorch 설치 정보 확인 (1.9.0 버전이 본 실습의 기본 설정)
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
# Detectron2 로깅(logging) 설정
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# 일반적으로 많이 사용되는 라이브러리 불러오기
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# 일반적으로 많이 사용되는 Detectron2 라이브러리 불러오기
from detectron2 import model_zoo # Detectron2 모델
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### <b>사전 학습된 Detectron2 모델 불러오기</b>


* Detectron2 환경설정(config) 정보를 생성한 뒤에 DefaultPredictor를 이용해 이미지에 대하여 추론합니다.
* [Detectron2 기본 환경설정 정보 확인하기](https://detectron2.readthedocs.io/en/latest/modules/config.html#config-references)
    * 참고로 cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST의 기본 값은 0.05입니다.

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# Detectron2 모델을 찾아 학습된 가중치 불러오기
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

model_final_280758.pkl: 167MB [00:04, 35.1MB/s]                           
The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [ ]:
outputs = predictor(im) # 이미지를 모델에 넣어 결과 계산하기

# 결과 출력하기
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# Visualizer를 이용해 이미지와 함께 예측 결과를 출력하기
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

### <b>데이터셋 등록하기</b>

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("my_coco_val2017_1000", {}, "my_coco_val2017_1000/annotations/instances.json", "my_coco_val2017_1000/data")

### <b>평가 진행하기</b>

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("my_coco_val2017_1000") # output_dir 인자 값이 없으면 오류 발생할 수 있음
val_loader = build_detection_test_loader(cfg, "my_coco_val2017_1000")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

#### <b>Test-Time Augmentation (TTA)</b>

* Detectron2에서의 [TTA 구현](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/test_time_augmentation.py)을 참고할 수 있습니다.
* Detectron2의 [modeling 라이브러리](https://detectron2.readthedocs.io/en/latest/modules/modeling.html#detectron2.modeling.GeneralizedRCNNWithTTA)를 참고할 수 있습니다.
* Detectron2의 [TTA 관련 질문](https://github.com/facebookresearch/detectron2/issues?q=TTA)을 참고할 수 있습니다.
* [Detectron2 기본 환경설정 정보 확인하기](https://detectron2.readthedocs.io/en/latest/modules/config.html#config-references)

In [ ]:
import copy
import numpy as np
from contextlib import contextmanager
from itertools import count
from typing import List
import torch
from fvcore.transforms import HFlipTransform, NoOpTransform
from torch import nn
from torch.nn.parallel import DistributedDataParallel

In [ ]:
from detectron2.config import configurable
from detectron2.data.detection_utils import read_image
from detectron2.data.transforms import (
    RandomFlip,
    ResizeShortestEdge,
    ResizeTransform,
    apply_augmentations,
)
from detectron2.structures import Boxes, Instances
from detectron2.modeling.meta_arch import GeneralizedRCNN
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import fast_rcnn_inference_single_image


# Test-Time Augmentation을 구현한 클래스
class MyDatasetMapperTTA:
    """
    이 클래스는 Detection 데이터셋을 입력으로 받고,
    config에 명시된 augmentation 기법이 적용된 이미지 리스트를 출력으로 내보냅니다.
    """

    @configurable
    def __init__(self, min_sizes: List[int], max_size: int, flip: bool):
        """
        [ResizeShortestEdge]: multi-scale 방법으로, 짧은 모서리를 기준으로 resize를 수행
            - min_sizes: list of short-edge size to resize the image to
            - max_size: maximum height or width of resized images
        [RandomFlip]: 좌우 반전(horizontal flip)이 기본 설정
            - flip: whether to apply flipping augmentation
        """
        self.min_sizes = min_sizes
        self.max_size = max_size
        self.flip = flip

    @classmethod
    def from_config(cls, cfg):
        return {
            "min_sizes": cfg.TEST.AUG.MIN_SIZES,
            "max_size": cfg.TEST.AUG.MAX_SIZE,
            "flip": cfg.TEST.AUG.FLIP,
        }

    def __call__(self, dataset_dict):
        """
       입력 이미지를 받고, augmentation 기법이 적용된 이미지를 내보냅니다.
            - 출력 dict 개수(기본 설정) = len(min_sizes) * (2 if flip else 1) = 18개
            - 각 dict는 "transforms" field를 포함하고, 해당 이미지를 만들 때 어떤 transforms (tfms)이 사용되었는지 기록
        """
        # transformation을 적용하기 위해 numpy 객체로 변환
        numpy_image = dataset_dict["image"].permute(1, 2, 0).numpy()
        shape = numpy_image.shape
        orig_shape = (dataset_dict["height"], dataset_dict["width"])
        if shape[:2] != orig_shape:
            # 원본 이미지를 input image로 변형하는 기본적인 함수(ResizeTransform)
            pre_tfm = ResizeTransform(orig_shape[0], orig_shape[1], shape[0], shape[1])
        else:
            pre_tfm = NoOpTransform()

        # 적용한 모든 augmentation의 조합(combination) 생성하기
        aug_candidates = [] # each element is a list[Augmentation] (augmentation들의 리스트 형태)
        for min_size in self.min_sizes:
            resize = ResizeShortestEdge(min_size, self.max_size)
            aug_candidates.append([resize]) # resize only
            if self.flip:
                flip = RandomFlip(prob=1.0)
                aug_candidates.append([resize, flip]) # resize + flip
        # (핵심) aug_candidates에 사용할 augmentations들을 리스트 형태로 삽입하면 구현 끝
        print(f'Augmentation 개수: {len(aug_candidates)} ', aug_candidates)

        # 모든 augmentation을 실제로 적용하기
        ret = []
        for aug in aug_candidates:
            # 입력 값: (연달아 적용할 augmentation들, 이미지 numpy 객체)
            # 반환 값: (augmentation을 적용한 이미지, 사용된 transforms 방법)
            new_image, tfms = apply_augmentations(aug, np.copy(numpy_image))
            torch_image = torch.from_numpy(np.ascontiguousarray(new_image.transpose(2, 0, 1))) # 다시 PyTorch 객체 형태로 변환

            dic = copy.deepcopy(dataset_dict)
            dic["transforms"] = pre_tfm + tfms # 사용한 transforms 내용
            dic["image"] = torch_image # 결과적으로 만들어진 학습 이미지
            ret.append(dic)
        return ret

In [ ]:
class GeneralizedRCNNWithTTA(nn.Module):
    """
    GeneralizedRCNN에 TTA를 적용하도록 해주는 클래스입니다. GeneralizedRCNN은 다음의 3단계로 구성됩니다.
        - GeneralizedRCNN: Per-image feature extraction (backbone) → Region proposal generation → Per-region feature extraction and prediction
    """

    def __init__(self, cfg, model, tta_mapper=None, batch_size=3):
        """
        (cfg, GeneralizedRCNN, tta_mapper, batch_size)를 입력으로 받습니다.
            - tta_mapper는 기본적으로 DatasetMapperTTA(cfg)를 사용합니다.
        """
        super().__init__()
        if isinstance(model, DistributedDataParallel):
            model = model.module
        assert isinstance(
            model, GeneralizedRCNN
        ), "TTA is only supported on GeneralizedRCNN. Got a model of type {}".format(type(model))
        self.cfg = cfg.clone()
        # 현재는 기본적인 RCNN 기반의 Object Detection에 최적화되어 있음
        assert not self.cfg.MODEL.KEYPOINT_ON, "TTA for keypoint is not supported yet"
        assert (
            not self.cfg.MODEL.LOAD_PROPOSALS
        ), "TTA for pre-computed proposals is not supported yet"

        self.model = model

        if tta_mapper is None:
            tta_mapper = DatasetMapperTTA(cfg) # 기본적으로 DatasetMapperTTA(cfg)를 사용(resize + flip)
        self.tta_mapper = tta_mapper
        self.batch_size = batch_size

    @contextmanager
    def _turn_off_roi_heads(self, attrs):
        """
        model.roi_heads에서 일시적으로 몇몇 head를 끄고, 그 상태의 context를 엽니다.
            - 예를 들어 "mask_on", "keypoint_on"과 같은 head를 끌 수 있습니다.
        """
        roi_heads = self.model.roi_heads
        old = {}
        for attr in attrs:
            try:
                old[attr] = getattr(roi_heads, attr)
            except AttributeError:
                # The head may not be implemented in certain ROIHeads
                pass

        if len(old.keys()) == 0:
            yield
        else:
            for attr in old.keys():
                setattr(roi_heads, attr, False)
            yield
            for attr in old.keys():
                setattr(roi_heads, attr, old[attr])

    def _batch_inference(self, batched_inputs, detected_instances=None):
        """
        입력 리스트에 대하여 추론(inference)을 수행합니다.
        GeneralizedRCNN.inference()와 동일한 입력/출력 형식을 갖습니다.
        """
        if detected_instances is None:
            detected_instances = [None] * len(batched_inputs)

        outputs = []
        inputs, instances = [], []
        for idx, input, instance in zip(count(), batched_inputs, detected_instances):
            inputs.append(input)
            instances.append(instance)
            if len(inputs) == self.batch_size or idx == len(batched_inputs) - 1:
                outputs.extend(
                    self.model.inference(
                        inputs,
                        instances if instances[0] is not None else None,
                        do_postprocess=False,
                    )
                )
                inputs, instances = [], []
        return outputs

    def __call__(self, batched_inputs):
        """
        GeneralizedRCNN.forward() 메서드와 동일한 input/output 형식을 가집니다.
        """
        def _maybe_read_image(dataset_dict):
            ret = copy.copy(dataset_dict)
            if "image" not in ret:
                image = read_image(ret.pop("file_name"), self.model.input_format)
                image = torch.from_numpy(np.ascontiguousarray(image.transpose(2, 0, 1)))  # CHW
                ret["image"] = image
            if "height" not in ret and "width" not in ret:
                ret["height"] = image.shape[1]
                ret["width"] = image.shape[2]
            return ret

        # 각 이미지에 대하여 _inference_one_image() 메서드를 적용한 결과 반환
        return [self._inference_one_image(_maybe_read_image(x)) for x in batched_inputs]

    def _inference_one_image(self, input):
        """
        Args:
            input (dict): one dataset dict with "image" field being a CHW tensor
        Returns:
            dict: one output dict
        """
        orig_shape = (input["height"], input["width"])
        augmented_inputs, tfms = self._get_augmented_inputs(input) # augmented 이미지를 확인
        # 모든 augmented 이미지에 대하여 bounding box를 예측
        with self._turn_off_roi_heads(["mask_on", "keypoint_on"]):
            # 일시적으로 roi head를 끈 상태로 예측 (bounding box를 얻음)
            all_boxes, all_scores, all_classes = self._get_augmented_boxes(augmented_inputs, tfms)
        # 바운딩 박스에 대한 최종 예측 결과를 얻기 위해, 모든 검출 박스를 합치기 (NMS 진행)
        merged_instances = self._merge_detections(all_boxes, all_scores, all_classes, orig_shape)

        if self.cfg.MODEL.MASK_ON:
            # Use the detected boxes to obtain masks
            augmented_instances = self._rescale_detected_boxes(
                augmented_inputs, merged_instances, tfms
            )
            # run forward on the detected boxes
            outputs = self._batch_inference(augmented_inputs, augmented_instances)
            # Delete now useless variables to avoid being out of memory
            del augmented_inputs, augmented_instances
            # average the predictions
            merged_instances.pred_masks = self._reduce_pred_masks(outputs, tfms)
            merged_instances = detector_postprocess(merged_instances, *orig_shape)
            return {"instances": merged_instances}
        else:
            return {"instances": merged_instances}

    def _get_augmented_inputs(self, input):
        augmented_inputs = self.tta_mapper(input)
        tfms = [x.pop("transforms") for x in augmented_inputs]
        return augmented_inputs, tfms

    def _get_augmented_boxes(self, augmented_inputs, tfms):
        # 1. 모든 augmented 이미지를 forward하여 결과를 얻기
        outputs = self._batch_inference(augmented_inputs)
        # 2. 얻은 결과 합치기(union)
        all_boxes = []
        all_scores = []
        all_classes = []
        for output, tfm in zip(outputs, tfms):
            # (핵심) 결과를 합칠 때는, box에 적용된 transforms을 inverse해야 original image에 대한 결과를 얻을 수 있음
            pred_boxes = output.pred_boxes.tensor
            original_pred_boxes = tfm.inverse().apply_box(pred_boxes.cpu().numpy())
            all_boxes.append(torch.from_numpy(original_pred_boxes).to(pred_boxes.device))

            all_scores.extend(output.scores)
            all_classes.extend(output.pred_classes)
        all_boxes = torch.cat(all_boxes, dim=0)
        return all_boxes, all_scores, all_classes

    def _merge_detections(self, all_boxes, all_scores, all_classes, shape_hw):
        # 모든 결과의 합(union)에서 선택
        num_boxes = len(all_boxes)
        num_classes = self.cfg.MODEL.ROI_HEADS.NUM_CLASSES
        # fast_rcnn_inference()은 background scores를 이용하므로, 1만큼 더해주기
        all_scores_2d = torch.zeros(num_boxes, num_classes + 1, device=all_boxes.device)
        for idx, cls, score in zip(count(), all_classes, all_scores):
            all_scores_2d[idx, cls] = score

        # fast_rcnn_inference_single_image()는 non-maximum suppression (NMS)을 진행하는 메서드임
        merged_instances, _ = fast_rcnn_inference_single_image(
            all_boxes,
            all_scores_2d,
            shape_hw,
            1e-8,
            self.cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST,
            self.cfg.TEST.DETECTIONS_PER_IMAGE,
        )

        return merged_instances

    def _rescale_detected_boxes(self, augmented_inputs, merged_instances, tfms):
        augmented_instances = []
        for input, tfm in zip(augmented_inputs, tfms):
            # Transform the target box to the augmented image's coordinate space
            pred_boxes = merged_instances.pred_boxes.tensor.cpu().numpy()
            pred_boxes = torch.from_numpy(tfm.apply_box(pred_boxes))

            aug_instances = Instances(
                image_size=input["image"].shape[1:3],
                pred_boxes=Boxes(pred_boxes),
                pred_classes=merged_instances.pred_classes,
                scores=merged_instances.scores,
            )
            augmented_instances.append(aug_instances)
        return augmented_instances

    def _reduce_pred_masks(self, outputs, tfms):
        # Should apply inverse transforms on masks.
        # We assume only resize & flip are used. pred_masks is a scale-invariant
        # representation, so we handle flip specially
        for output, tfm in zip(outputs, tfms):
            if any(isinstance(t, HFlipTransform) for t in tfm.transforms):
                output.pred_masks = output.pred_masks.flip(dims=[3])
        all_pred_masks = torch.stack([o.pred_masks for o in outputs], dim=0)
        avg_pred_masks = torch.mean(all_pred_masks, dim=0)
        return avg_pred_masks

* Basic Method

In [ ]:
my_dataset_mappter_tta = MyDatasetMapperTTA(
    min_sizes=cfg.TEST.AUG.MIN_SIZES,
    max_size=cfg.TEST.AUG.MAX_SIZE,
    flip=cfg.TEST.AUG.FLIP,
)
tta_model = GeneralizedRCNNWithTTA(cfg, predictor.model, tta_mapper=my_dataset_mappter_tta)

In [ ]:
print(inference_on_dataset(tta_model, val_loader, evaluator))

* 다음의 기능들을 사용할 수 있는 형태로 재구현
    * RandomBrightness
    * RandomContrast
    * RandomFlip
    * RandomSaturation
    * RandomRotation
    * ResizeShortestEdge

In [ ]:
from detectron2.config import configurable
from detectron2.data.detection_utils import read_image
from detectron2.data.transforms import (
    RandomBrightness,
    RandomContrast,
    RandomFlip,
    RandomSaturation,
    RandomRotation,
    ResizeShortestEdge,
    ResizeTransform,
    apply_augmentations,
)
from detectron2.structures import Boxes, Instances
from detectron2.modeling.meta_arch import GeneralizedRCNN
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import fast_rcnn_inference_single_image


# Test-Time Augmentation을 구현한 클래스
class MyDatasetMapperTTA:
    """
    이 클래스는 Detection 데이터셋을 입력으로 받고,
    config에 명시된 augmentation 기법이 적용된 이미지 리스트를 출력으로 내보냅니다.
    """
    @configurable
    def __init__(self, selected: List[int], min_sizes: List[int], max_size: int, flip: bool):
        """
        [RandomBrightness]: 랜덤으로 명도(brightness) 변경
            - intensity_min: minimum augmentation
            - intensity_max: maximum augmentation
        [RandomContrast]: 랜덤으로 대조(contrast) 변경
            - intensity_min: minimum augmentation
            - intensity_max: maximum augmentation
        [RandomFlip]: 좌우 반전(horizontal flip)이 기본 설정
            - flip: whether to apply flipping augmentation
        [RandomSaturation]: 랜덤으로 채도(saturation) 변경
            - intensity_min: minimum augmentation
            - intensity_max: maximum augmentation
        [RandomRotation]: 랜덤으로 시계 방향 회전(rotation) 수행
            - angle: [min, max] interval from which to sample the angle (in degrees)
        [ResizeShortestEdge]: multi-scale 방법으로, 짧은 모서리를 기준으로 resize를 수행
            - min_sizes: list of short-edge size to resize the image to
            - max_size: maximum height or width of resized images
        """
        self.selected = selected
        self.min_sizes = min_sizes
        self.max_size = max_size
        self.flip = flip

        print('[Your selection]')
        for i in range(6):
            if i == 0 and self.selected[i] == True:
                print('RandomBrightness')
            if i == 1 and self.selected[i] == True:
                print('RandomContrast')
            if i == 2 and self.selected[i] == True:
                print('RandomFlip')
            if i == 3 and self.selected[i] == True:
                print('RandomSaturation')
            if i == 4 and self.selected[i] == True:
                print('RandomRotation')
            if i == 5 and self.selected[i] == True:
                print('ResizeShortestEdge')

    @classmethod
    def from_config(cls, cfg):
        return {
            "min_sizes": cfg.TEST.AUG.MIN_SIZES,
            "max_size": cfg.TEST.AUG.MAX_SIZE,
            "flip": cfg.TEST.AUG.FLIP,
        }

    def __call__(self, dataset_dict):
        """
       입력 이미지를 받고, augmentation 기법이 적용된 이미지를 내보냅니다.
            - 출력 dict 개수(기본 설정) = len(min_sizes) * (2 if flip else 1) = 18개
            - 각 dict는 "transforms" field를 포함하고, 해당 이미지를 만들 때 어떤 transforms (tfms)이 사용되었는지 기록
        """
        # transformation을 적용하기 위해 numpy 객체로 변환
        numpy_image = dataset_dict["image"].permute(1, 2, 0).numpy()
        shape = numpy_image.shape
        orig_shape = (dataset_dict["height"], dataset_dict["width"])
        if shape[:2] != orig_shape:
            # 원본 이미지를 input image로 변형하는 기본적인 함수(ResizeTransform)
            pre_tfm = ResizeTransform(orig_shape[0], orig_shape[1], shape[0], shape[1])
        else:
            pre_tfm = NoOpTransform()

        aug_candidates = [[]] # each element is a list[Augmentation] (augmentation들의 리스트 형태)
        # 적용한 모든 augmentation의 조합(combination) 생성하기
        for i in range(6):
            if i == 0 and self.selected[i] == True:
                temp = []
                for _ in range(3):
                    brightness = RandomBrightness(intensity_min=0.8, intensity_max=1.2)
                    for aug in aug_candidates:
                        temp.append(aug + [brightness])
                aug_candidates = aug_candidates + temp
            if i == 1 and self.selected[i] == True:
                temp = []
                for _ in range(3):
                    contrast = RandomContrast(intensity_min=0.8, intensity_max=1.2)
                    for aug in aug_candidates:
                        temp.append(aug + [contrast])
                aug_candidates = aug_candidates + temp
            if i == 2 and self.selected[i] == True:
                flip = RandomFlip(prob=1.0)
                temp = []
                for aug in aug_candidates:
                    temp.append(aug + [flip])
                aug_candidates = aug_candidates + temp
            if i == 3 and self.selected[i] == True:
                temp = []
                for _ in range(3):
                    saturation = RandomSaturation(intensity_min=0.8, intensity_max=1.2)
                    for aug in aug_candidates:
                        temp.append(aug + [saturation])
                aug_candidates = aug_candidates + temp
            if i == 4 and self.selected[i] == True:
                temp = []
                for _ in range(3):
                    rotation = RandomRotation(angle=[-10, 10])
                    for aug in aug_candidates:
                        temp.append(aug + [rotation])
                aug_candidates = aug_candidates + temp
            if i == 5 and self.selected[i] == True:
                temp = []
                for min_size in self.min_sizes:
                    resize = ResizeShortestEdge(min_size, self.max_size)
                    for aug in aug_candidates:
                        temp.append(aug + [resize])
                aug_candidates = aug_candidates + temp
        # (핵심) aug_candidates에 사용할 augmentations들을 리스트 형태로 삽입하면 구현 끝
        print(f'Augmentation 개수: {len(aug_candidates)} ', aug_candidates)

        # 모든 augmentation을 실제로 적용하기
        ret = []
        for aug in aug_candidates:
            # 입력 값: (연달아 적용할 augmentation들, 이미지 numpy 객체)
            # 반환 값: (augmentation을 적용한 이미지, 사용된 transforms 방법)
            new_image, tfms = apply_augmentations(aug, np.copy(numpy_image))
            torch_image = torch.from_numpy(np.ascontiguousarray(new_image.transpose(2, 0, 1))) # 다시 PyTorch 객체 형태로 변환

            dic = copy.deepcopy(dataset_dict)
            dic["transforms"] = pre_tfm + tfms # 사용한 transforms 내용
            dic["image"] = torch_image # 결과적으로 만들어진 학습 이미지
            ret.append(dic)
        return ret

In [ ]:
my_dataset_mappter_tta = MyDatasetMapperTTA(
    # RandomBrightness, RandomContrast, RandomFlip, RandomSaturation, RandomRotation, ResizeShortestEdge
    [True, True, True, False, False, False],
    min_sizes=cfg.TEST.AUG.MIN_SIZES,
    max_size=cfg.TEST.AUG.MAX_SIZE,
    flip=cfg.TEST.AUG.FLIP,
)
tta_model = GeneralizedRCNNWithTTA(cfg, predictor.model, tta_mapper=my_dataset_mappter_tta)

[Your selection]
RandomBrightness
RandomContrast
RandomFlip


In [ ]:
print(inference_on_dataset(tta_model, val_loader, evaluator))